# Dataset creation

## Dependencies and imports

In [ ]:
import os
import re
import json
import time

import pandas as pd
from openai import OpenAI

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
PROMPT = 'Semplifica questo testo in maniera che sia comprensibile e chiaro per una persona con la terza media:'

Mounted at /content/drive


## Load raw dataset

In [ ]:
corpus = pd.read_excel(f'./corpus/corpus.xlsx')
corpus.head()

,topic,region,document_type,document_ref,original_text
0,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,La Carta dei servizi è uno strumento operativo...
1,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,Il Comune di Bernalda ha elaborato questa Cart...
2,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,"I criteri della Carta dei Servizi, sono da con..."
3,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,I Servizi erogati nel Comune di Bernalda sono ...
4,garbage,basilicata,service_charter,a958e53905ca50a2b93e2a1725e08fec,Nessuna discriminazione nell'erogazione delle ...


In [ ]:
dataset = []
if os.path.exists(f'./dataset/dataset.json'):
  dataset = json.load(open(f'./dataset/dataset.json', 'r', encoding='utf-8'))
else:
  dataset = corpus.to_dict(orient='records')

## OpenAI simplification functions

In [ ]:
client = OpenAI(
  api_key=os.getenv('OPENAI_API_KEY')
)

In [ ]:
def simplify_openai(message):
  try:
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      messages=[
        {
          "role": "system",
          "content": PROMPT
        },
        {
          "role": "user",
          "content": message
        }
      ],
      temperature=0.7,
      max_tokens=258,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    # print(response)
    text = response.choices[0].message.content
    return text
  except Exception as e:
    print(e)
    return ''

## Few examples

In [ ]:
input = '''
La Carta dei servizi è uno strumento operativo voluto da una direttiva della Presidenza del Consiglio dei Ministri del 27 gennaio 1994 che definisce i principi che devono regolare la fornitura dei servizi pubblici, individuando gli strumenti per l'attuazione e le modalità per la tutela dei cittadini-utenti.
Il Comune di Bernalda ha elaborato questa Carta dei Servizi fissando criteri di prestazione e standard di qualità degli stessi prevedendo idonei strumenti di controllo e di verifica per la tutela dei propri clienti. I criteri della Carta dei Servizi, sono da considerare validi in condizioni normali di esercizio e non in situazioni straordinarie dovute ad eventi naturali eccezionali, eventi causati da terzi, scioperi (diretti o indiretti) o atti dell'Autorità pubblica. I Servizi erogati nel Comune di Bernalda sono gestiti nel rispetto dei seguenti principi generali previsti dalla direttiva del Presidente del Consiglio dei Ministri del 27.01.1994 in materia di erogazione dei servizi pubblici.
'''

print(simplify_openai(input))

La Carta dei servizi è un documento che indica come devono essere forniti i servizi pubblici e come i cittadini devono essere protetti. Il Comune di Bernalda ha creato questa carta per garantire che i servizi offerti siano di buona qualità e prevede controlli per proteggere i cittadini. I criteri della Carta valgono in condizioni normali, ma non in situazioni straordinarie come eventi naturali eccezionali, scioperi o atti dell'autorità pubblica. I servizi offerti nel Comune di Bernalda devono rispettare i principi stabiliti dalla direttiva del Presidente del Consiglio dei Ministri del 27 gennaio 1994.


In [ ]:
input = '''
In caso di rettifica di fatturazione per le bollette pagate:
- se dovuta ad errori di fatturazione, ASA SpA provvederà alla comunicazione dell’esito della rettifica entro i tempi previsti di cui sopra;
- se dovute a richieste di verifica del contatore, si procederà secondo quanto previsto al successivo Art.4.3.4 Verifica del contatore.
'''

print(simplify_openai(input))

Se hai pagato una bolletta e c'è un errore nella fatturazione, ASA SpA ti informerà sulle correzioni entro i tempi previsti. Se l'errore è legato a una richiesta di verifica del contatore, verrà seguito quanto indicato nell'articolo successivo riguardante la verifica del contatore.


In [ ]:
input = '''
Uno dei fattori chiave per la tutela e promozione della salute individuale e di comunità è quello di creare comunità resilienti e ambienti favorevoli, in grado di reagire a situazioni nuove o avverse. Appare dunque essenziale responsabilizzare prima di tutto chi è alla guida dell'amministrazione nelle scelte allocative, poi coloro che operano nelle strutture sanitarie e infine i cittadini nell'adozione di corretti stili di vita al fine anche di ridurre il carico prevedibile e prevenibile di malattie croniche non trasmissibili.
'''

print(simplify_openai(input))

Per proteggere la salute delle persone e delle comunità, è importante creare ambienti e comunità che siano forti e capaci di affrontare nuove situazioni difficili. È fondamentale che chi guida l'amministrazione, chi lavora negli ospedali e i cittadini stessi si sentano responsabili di fare scelte che favoriscano la salute. In questo modo si può anche ridurre il rischio di malattie croniche non trasmissibili.


In [ ]:
input = '''
Il servizio di medicina generale assicura:
- Le prestazioni di base;
- Le urgenze cliniche;
- Il servizio accoglienza nuovi giunti;
- Certificazioni mediche in uso nell'ambito penitenziario;
- Le prestazioni medico-legali a favore della Polizia Penitenziaria e della popolazione detenuta in caso di assenza del responsabile e non procrastinabili.
'''

print(simplify_openai(input))

Il servizio di medicina generale offre:
- Cura di base per problemi di salute comuni;
- Assistenza medica urgente;
- Accoglienza e assistenza per nuovi pazienti;
- Certificati medici per i detenuti;
- Assistenza medica legale per la Polizia Penitenziaria e i detenuti in casi urgenti.


## Expected pricing

In [ ]:
# gpt-3.5-turbo
# input cost: $0.0010 / 1K tokens
# output cost: $0.0015 / 1K tokens

tokens = 0
for sample in dataset:
  tokens += len(sample['original_text'].split(' '))

input_cost = tokens / 1000 * 0.0010
output_cost = tokens / 1000 * 0.0015

print('Potential cost: ', input_cost + output_cost)
print('Potential cost with tax: ', (input_cost + output_cost) * 1.22)

Potential cost:  2.03763
Potential cost with tax:  2.4859086


## Do simplification

In [ ]:
for sample in tqdm(dataset):
  if 'oracle_text' not in sample or sample['oracle_text'] == '':
    oracle_text = simplify_openai(sample['original_text'])
    sample['oracle_text'] = oracle_text

100%|██████████| 18349/18349 [06:27<00:00, 47.31it/s]


In [ ]:
dataset_file = open(f'./dataset/dataset.json', 'w')
json.dump(dataset, dataset_file, indent=2)
dataset_file.close()

## Saving full dataset

In [ ]:
dataset_df = pd.DataFrame(dataset)
dataset_df.sample(20)

,topic,region,document_type,document_ref,original_text,oracle_text
2535,public_services,calabria,public_holdings_rationalization,92f810e5aab710916f6d1bbfb6b7513c,In riferimento al mantenimento delle partecipa...,La Regione ha l'obbligo di spiegare perché tie...
16080,garbage,veneto,tender,067d8e49edfee9eecae02a85365bf1f3,L'appalto ha ad oggetto l'affidamento del serv...,"L'appalto riguarda un servizio di raccolta, tr..."
6769,healthcare,lombardia,accreditation,df9cf30ce0fbfabd587d31e67674bc6d,Il diario riporta le motivazioni a supporto di...,Il diario spiega perché certe prestazioni non ...
483,healthcare,basilicata,planning_acts,99bdc9fdd8097f067f77cb220074b1b5,Il telecontrollo si affianca idealmente al tel...,Il telecontrollo e il telemonitoraggio dovrebb...
17222,healthcare,veneto,planning_acts,cca682c8e597276a9313e6eff21fe065,garantire il prioritario intervento sull'età g...,Assicurare che si dia priorità all'aiuto ai gi...
10519,healthcare,molise,accreditation,69d186bf1e2e397e771552a96c49d10b,Il gruppo di esperti individuato per la proced...,Il gruppo di esperti scelto per fare un'ispezi...
7314,public_services,lombardia,service_charter,7b74941fd51f8395790873b01e75881e,Il Centro Socio Educativo (C.S.E.) è un'Unità ...,Il Centro Socio Educativo (C.S.E.) è un serviz...
6153,healthcare,lazio,accreditation,d4b9addca250f5eac44d42a131c67f29,vi è evidenza della rivalutazione e della revi...,Quando le condizioni di salute di un paziente ...
15350,public_services,toscana,public_holdings_rationalization,1be983abb88fd603ee16576df2caa899,"La società, come si rileva dall'art.4 dello st...","La società, come si legge nell'articolo 4 dell..."
762,public_services,basilicata,public_holdings_rationalization,f367e0027ea593308875913e1e8547c1,"L'esito della ricognizione, anche in caso nega...","I risultati della verifica, anche se non trovi..."


In [ ]:
dataset_df.shape

(18349, 6)

In [ ]:
dataset_df.to_csv(f'./dataset/dataset.tsv', index=False, sep='\t')
dataset_df.to_excel(f'./dataset/dataset.xlsx', index=False)

## Train, test and val splitting

![train-test-val.jpg](https://miro.medium.com/v2/resize:fit:1400/1*f2KznlrIdj1MeobprVGBtg.png)

In [ ]:
train_df, rem_df = train_test_split(dataset_df, train_size=0.8)
test_df, val_df = train_test_split(rem_df, train_size=0.5)

In [ ]:
train_df.to_csv(f'./dataset/train.tsv', index=False, sep='\t')
test_df.to_csv(f'./dataset/test.tsv', index=False, sep='\t')
val_df.to_csv(f'./dataset/val.tsv', index=False, sep='\t')